#requirements
earthengine-api
geemap

token: 4/1AQlEd8w_NhJ7mk47qLkoAjWcWgJWyyI-I_eFvnRNVmr5cmrs-ghCWkrvUMY (only lasts a week)

In [4]:
import ee
# Authenticate the Earth Engine API
ee.Authenticate()

# Initialize the Earth Engine API
ee.Initialize()



Successfully saved authorization token.


In [6]:
import geemap

def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-01-01', '2020-01-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(83.277, 17.7009, 12)
m.add_layer(dataset.mean(), visualization, 'RGB')
m

Map(center=[17.7009, 83.277], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [8]:
# Define the region of interest (ROI) - Bounding box for England
roi = ee.Geometry.Rectangle([-7.572168, 49.959999, 1.681530, 58.635000])

# Load the Sentinel-2 SR Harmonized Image Collection
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterDate('2023-01-01', '2023-12-31') \
    .filterBounds(roi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))  # Filter by cloud cover

# Select the bands you are interested in (e.g., RGB)
bands = ['B4', 'B3', 'B2']

# Get the median image (or use another method to combine images)
median_image = collection.median().select(bands)

# Visualization parameters optimized for high resolution
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': bands,
    'gamma': 1.4  # Adjust gamma to improve contrast
}

# Initialize the geemap map, centered on England, with an appropriate zoom level
Map = geemap.Map(center=[52.3555, -1.1743], zoom=8)  # Adjusted zoom level to be closer to the native resolution

# Add the median image to the map
Map.addLayer(median_image, vis_params, "Sentinel-2 Max Resolution")

# Display the map with layer control
Map.addLayerControl()
Map


Map(center=[52.3555, -1.1743], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…